In [ ]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import pdfplumber   

from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [21]:
# Grundläggande initieringar
load_dotenv()
genai.configure(api_key=os.getenv("API_KEY"))
model = genai.GenerativeModel(model_name="models/gemini-1.5-flash-latest")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
INDEX_PATH = "mitt_index"           # Sökväg till FAISS-indexmapp


# RAG

In [22]:
# Hämta PDF-filer
def get_all_pdfs_in_folder(folder_path):
    return [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if file.endswith(".pdf")
    ]
    
# Ladda PDF:er som sidor med sidunummer och dokumentnamn som metadata
def load_clean_pdfs_with_page_metadata(pdf_paths):
    docs = []
    for pdf_path in pdf_paths:
        with pdfplumber.open(pdf_path) as pdf:              # använder pdfplumber för bättre hantering av rubriker, punktlistor, kolumner m.m som är vanliga i rapporterna... istället för att platta ut..
            filename = os.path.basename(pdf_path)
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text:
                    clean_text = " ".join(text.split())     # städar i texten
                    metadata = {"source": filename, "page": i + 1}
                    docs.append(Document(page_content=clean_text, metadata=metadata))
    return docs


In [23]:
# Chunkar dokument
def chunk_documents(docs, chunk_size=700, chunk_overlap=150): # ganska stora chunks och overlap för att inte missa kontext.
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " "]     # hirearkisk lista över var text helst ska brytas.
    )
    return splitter.split_documents(docs)       # metadata behålls

# Skapar FAISS-index med embeddings
def store_in_faiss(chunks):
    return FAISS.from_documents(chunks, embeddings)

In [24]:
# Frågar Gemini med kontext från FAISS
def ask_with_gemini(faiss_index, query, k=12, max_words=100):   
    matched_docs = faiss_index.similarity_search(query, k=k)    # hittar k chunks baserat på cosine similarity på embeddings.

    # Skapar kontextsträng med källa + sida
    context = "\n\n".join([
        f"Från {doc.metadata['source']} (sida {doc.metadata['page']}):\n{doc.page_content.strip()}"
        for doc in matched_docs
    ])
    
    # Skapar prompt med kontext och fråga
    system_prompt = (
    "Svara utifrån följande sammanställningar av forskningsrapporter:\n\n"
    f"{context}\n\n"
    "❗ Analysera texten noggrant. Ignorera ofullständiga meningar, sidhuvuden, figurer och punktlistor.\n"
    f"Svara på frågan: {query}\n"
    f"Begränsa dig till max {max_words} ord och ange sidnummer och dokument i svaret."
    )
    
    response = model.generate_content(system_prompt)
    return response.text, system_prompt

In [29]:
# Användning

# Ladda eller bygg FAISS-index
if os.path.exists(f"{INDEX_PATH}/index.faiss") and os.path.exists(f"{INDEX_PATH}/index.pkl"):
    print("Laddar befintligt FAISS-index...")
    faiss_index = FAISS.load_local(INDEX_PATH, embeddings, allow_dangerous_deserialization=True)

else:
    print("Skapar nytt FAISS-index...")
    folder = r"KKV"
    pdf_files = get_all_pdfs_in_folder(folder)
    docs_text = load_clean_pdfs_with_page_metadata(pdf_files)
    chunks = chunk_documents(docs_text)
    faiss_index = store_in_faiss(chunks)
    faiss_index.save_local(INDEX_PATH)
    print("FAISS-index sparat.")

# Testfråga:
svar, prompt = ask_with_gemini(faiss_index, "finns det kokurrensproblem i matkedjan?", max_words=150)


print("Svar:\n", svar)

Laddar befintligt FAISS-index...
Svar:
 Ja, det finns flera konkurrensproblem i matkedjan.  Bristande transparens och orättvisa kontrollavgifter påverkar konkurrensen i primärproduktionen (Konkurrensen i primärproduktionen, s. 19).  Dagligvaruhandelns höga priser överstiger ofta kostnadsökningar (Stigande matpriser och lönsamhet, s. 26).  Komplexa regelverk gynnar etablerade aktörer (Dagligvaruhandelns etablering, s. 16), och begränsningar i avtal mellan slakterier och uppfödare försvårar byte av köpare (Avtalsstrukturer i livsmedelskedjan, s. 16).  Dessutom kan tidsfönster för lansering av varor skapa svårigheter för leverantörer (Tidsfönster för lansering av varor, s. 19).  Konkurrensverket har identifierat flera av dessa problem.



In [28]:
# Kontroll av laddade dokument
print(f"{len(pdf_files)} dokument laddade:")
for filepath in pdf_files:
    filename = os.path.basename(filepath)
    print(f" - {filename}")

12 dokument laddade:
 - Alternativa distributionsformer.pdf
 - Avtalsstrukturer i livsmedelskedjan.pdf
 - Dagligvaruhandelns etablering.pdf
 - Handelns egna varumärken.pdf
 - Hinder för konkurrens.pdf
 - Konkurrensen dagligvaruaktörer.pdf
 - Konkurrensen i primärproduktionen.pdf
 - Konkurrensen i äggkedjan.pdf
 - Konkurrensverkets sammanfattning.pdf
 - Stigande matpriser och lönsamhet.pdf
 - Tidsfönster för lansering av varor.pdf
 - Ökande livsmedelspriser och konkurrens.pdf


# Självevaluering

In [30]:
validation_data = [
    {
        "question": "Vart har priset för ägg ökat mest",
        "ideal_answer": "Det vi funnit är att priset ökat mest för ägg från frigående höns inomhus som säljs av stormarknader. "
    },
    {
        "question": "Vad avses med begreppet 'tidsfönster' inom dagligvaruhandeln?",
        "ideal_answer": "tidsfönstren avser de tidpunkter när förändringar i produkt sortimentet bör ske i butiksledet, antingen genom nylansering av en produkt eller genom revidering av en befintlig produkt, exempelvis med anledning av en nsmak eller en större förpackning. Det kan också avse tidpunkter för utfasning av produkter."
    }, 
    {
        "question": "När dog Olof Palme?",
        "ideal_answer": "Jag har inte tillgång till information för att svara på den frågan." 
    }
 ]

In [31]:
 evaluation_system_prompt ="""Du är ett intelligent neutralt utvärderingssystem vars uppgift är att utvärdera en AI-assistents svar. 
 Om svaret är väldigt nära det önskade svaret, sätt poängen 1. Om svaret är felaktigt eller inte bra nog,sätt poängen 0. 
 Om svaret innehåller rätt information, men även mer info som komplementerar den i idealsvaret, kan du sätta 1.
 Om svaret är delvis i linje med det önskade svaret, sätt poängen 0.5. Motivera kort varför du sätter den poäng du gör.
 """

In [32]:
def evaluate_response(faiss_index, question, ideal_answer):
    # Få svar från modellen
    response_text, _ = ask_with_gemini(faiss_index, question)

    # Skapa utvärderingsprompt
    evaluation_prompt = f"""{evaluation_system_prompt}

Fråga: {question}
AI-assistentens svar: {response_text}
Önskat svar: {ideal_answer}"""

    # Skicka prompt till modellen direkt, utan FAISS denna gång
    evaluation_response = model.generate_content(evaluation_prompt)

    return response_text, evaluation_response.text


In [33]:
def extract_score(evaluation_text):
    
    for token in ["1", "0.5", "0"]:
        if f"Poäng: {token}" in evaluation_text or f"poängen {token}" in evaluation_text:
            return float(token)
    return 0            # default om inget kan tolkas

In [34]:
def run_full_evaluation(validation_data, faiss_index):
    results = []
    total_score = 0

    for i, item in enumerate(validation_data):
        ai_response, evaluation_text = evaluate_response(faiss_index, item["question"], item["ideal_answer"])
        score = extract_score(evaluation_text)

        results.append({
            "question": item["question"],
            "ai_answer": ai_response,
            "ideal_answer": item["ideal_answer"],
            "evaluation": evaluation_text,
            "score": score
        })

        total_score += score

    average_score = total_score / len(validation_data)

    # Sammanfattning
    print("\n====== Utvärdering av valideringsfrågor ======")
    for i, r in enumerate(results):
        print(f"\n --> Fråga {i+1}: {r['question']}")
        print(f" AI-svar: {r['ai_answer']}")
        print(f" Idealsvar: {r['ideal_answer']}")
        print(f" Utvärdering: {r['evaluation']}")

    print(f"\n=> Genomsnittligt betyg: {round(average_score, 2)} av 1.0 <=")

# Kör allt
run_full_evaluation(validation_data, faiss_index)



====== Utvärdering av valideringsfrågor ======

 --> Fråga 1: Vart har priset för ägg ökat mest
 AI-svar: Priset på frigående inomhusegg har ökat mest på stormarknader jämfört med kvartersbutiker.  Detta framgår av "Konkurrensen i äggkedjan.pdf", sida 16.  Dokumenten nämner även att priset på ägg generellt ökat mer än andra livsmedel, men specificerar inte var ökningen varit störst utöver just frigående inomhusegg på stormarknader.

 Idealsvar: Det vi funnit är att priset ökat mest för ägg från frigående höns inomhus som säljs av stormarknader. 
 Utvärdering: Poäng: 1

Motivering: AI-assistentens svar matchar det önskade svaret nästan perfekt.  Informationen är identisk och tillägget om att äggpriset generellt ökat mer än andra livsmedel är relevant och kompletterande information som stärker svaret utan att vara vilseledande eller irrelevant.  Källhänvisningen är också en positiv aspekt.


 --> Fråga 2: Vad avses med begreppet 'tidsfönster' inom dagligvaruhandeln?
 AI-svar: "Tidsfönst

# Fördjupad diskussion

Min RAG är tränad på 11 rapporter, samt en sammanfattande rapport, kring konkurrenssituationen inom matbranschen. Rapporterna är framtagna av, eller på uppdrag av, Konkurrensverket inom ramen av ett regeringsuppdrag.

Att träna en språkmodell på given input är bra i olika kontexter där man vill vara säker vilken data svaren som modellen genererar är baserade på, och där man inte är intresserad av annan input.
I detta fall skulle Konkurrensverket i samband med publicering av rapporterna också ha kunnat publicera en chatbot som svarade på frågor kring dessa specifika rapporter. Det kan vara till nytta för allmänheten, skolelever, forskare eller  politiker som är intresserade av en specifik frånga, eller vill ha en snabb överblick. Genom att lägga til rapport- och sidhänvisning i modellens svar, som jag gjort, kan den intresserade snabbt ta sig till den rapport som berör det som är av intresse. Detta torde i många fall vara mer effektivt än att användaren ska vara hänvisad till läsning av sammanfattningar, vilket även de i många fall är långa texter som kan kännas omständiga för den som vill ha ett snabbt svar.

Användningsområdena är många för denna typ av modeller; regler och lagar, utbildningmaterial, policys, kontrakt och avtal, instruktioner, databaser med rättsfall för snabb sökning av praxis etc. 

Fördelarna är två, varav den ena redan är nämnd; Modellen svarar utifrån den data du vill och uppger (om instruerade att göra så) att den inte har information för att svara om man frågar sådant som inte finns i materialet. Detta i kontrast till en generell modell, vilken "chansar" om fakta saknas.

Detta kan vara positivt då användarna kan få en snabb och kostnadseffektiv tillgång till fakta och får ett faktabaserat underlag till beslut - utifrån det givna underlaget, utan gissningar och input från andra källor.

Den andra fördelen är att du kan träna modellen på konfidentiell information som du inte vågar tillgängliggöra till en öppen allmän språkmodell. Genom RAG ges möjlighet att den språkmodell som nyttjas tar del av det känsliga materialet i en sluten miljö. Metoden i sig är dock inte säker per se men möjliggör sekretess genom kryptering av, och åtkomstkontroll till, databasen med vektorer.

Utöver att säkerställa önskad informationssäkerhet finns andra problemområden värda att känna till med RAG-modeller.

- Kvalitén på de svar som modellen ger blir aldrig bättre än kvalitén på det inläsat materialet. Är den felaktig, utdaterade, ofullständigt eller vinklad blir modellens svar också det. Att säkerställa relevant och korrekt information som modellen får läsa in sig på är av största vikt!

- Även om input-data är korrekt är inte RAG modeller perfekta. Även om de inte gissar så finns sannolikheten att de missar viss information som är relevant. Chunkning och modellspecifikation är viktiga att mixtra med. T.ex. ställde jag till en början in att modellen skulle titta i relativt få kontextuella chunkar när den konstruerade svar. I och med att jag hade ganska många dokument (12) missades vid vissa frågor input från vissa dokument.

- Givet ovan potentiella tillkortakommanden är det också intressant att fundera på vem som är ansvariga när ett dåligt beslut fattas utifrån dålig information från modellen. Den som programmerat modellen dåligt? Den som är ansvarig för att tillse att inputdata är av god kvalité? Användaren som borde dubbelkolla faktan i de svar som modellen genererar?

- Jag kan också se att det finns en risk i case med komplex och omfattande information - som exempelvis i mitt fall med många forskningsrapporter - att den som ställer en fråga får ett väl koncist svar, är nöjd med det, och inte ger sig tid att läsa rapporten för att förstå den helhet och det större perspektiv som krävs för djupförståelse. Lite som att läsa en korrekt men kort artikel om ett krig, och utifrån den välja sida för sina sympatier i konflikten.

Sammanfattningsvis tänker jag att RAG-modeller kan vara till stor nytta och skapa stora effektivitetsvinster. Men att man måste nyttja dem med eftertanke och anpassa hantering av den information man får från modellen utifrån karaktären/vikten av de beslut/handlingar som de leder till.



# Självutvärdering

 #### 1. Vad har varit roligast i kunskapskontrollen?
 Kul att se att det ändå är rätt snabbt gjort att skapa något som många företag och organisationer skulle ha ganska stor nytta av! Mer specifikt så tror jag att den exempelanvändning som jag programmerat skulle kunna vara av nytta för myndigheter
 och forskningsinstutiotioner som gärna vill göra information från ofta tunga rapporter och utredningar lätt tillgängliga för den stora allmänheten.

 #### 2. Vilket betyg anser du att du ska ha och varför?
 Jag har gått för VG och tycker jag levererar enligt de uppställda kriterierna.

 
 #### 3. Vad har varit mest utmanande i arbetet och hur har du hanterat det?
Jag hade svårt att komma på vad för information jag skulle ha som input i min modell. Alla idéer jag hade krävde information från företag som inte är offentlig. Jag googlade runt, diskuterade med ChatGPT och kom sedan på idén med Konkurrensverkets rapporter i samband med att jag var på deras hemsida för att reka inför ansökan om LIA. 
